In [ ]:
from nltk import edit_distance
from itertools import product, combinations
import csv
from joblib import Parallel, delayed

In [ ]:
D = []
IdNum = 354
i = 0
for x,y in product([pairMatches[pairMatches.yearId ==2]['m_desc'].iloc[IdNum]], pairMatches[pairMatches.yearId == 4]['m_desc']):
    i += 1
    
    if i % 1000 == 0:
        print 'I = {}\r'.format(i),
    D.append(edit_distance(x,y))

In [ ]:
def runDistanceAnalysis(yrs,mf):
    
    yr1, yr2 = yrs
    
    if mf == 'm':
        survey = 'motherSurvey'
    else:
        survey = 'fatherSurvey'
    
    idx_1 = pairMatches[pairMatches.yearId==yr1][survey].values
    idx_2 = pairMatches[pairMatches.yearId==yr2][survey].values
    
    idx_list = product(idx_1,idx_2)
    i = 0
    D = []
    for x,y in idx_list:
        i += 1

        if i % 1000 == 0:
            print '{}-Year{}-Year{} = {}\r'.format(mf,yr1,yr2,i),

        desc_x = pairMatches[pairMatches[survey]== x]['{}_desc'.format(mf)].values[0]
        desc_y = pairMatches[pairMatches[survey]== y]['{}_desc'.format(mf)].values[0]
        D.append(('{}:{}'.format(x,y),edit_distance(desc_x,desc_y)))
            
    with open('../output/{}_survey_year{}-year{}.csv'.format(mf,yr1,yr2), 'wb') as wFile:
        csv_out=csv.writer(wFile)
        csv_out.writerow(['relation','distance score'])
        for row in D:
            csv_out.writerow(row)
    
    return True
    
yrs = [1,2,3,4,5]
yr_list = combinations(yrs,2)
in_para = product(yr_list,['m', 'f'])

Parallel(n_jobs=20)(delayed(runDistanceAnalysis)(yrs=yrs,mf=mf) for yrs,mf in in_para)